In [2]:
import itertools
import pandas as pd
from kanren import *

In [8]:
room_data = pd.read_csv('usprc.csv')
room_data.head(1)

,Entry ID,Gender Description,Classification Description 9,Room Preference Description 1,Room Preference Description 2,Room Preference Description 3,Room Preference Comments,Floor Gender Preference,Nationality Description 4,PR Status,Admit Term,Enrollment Term,Faculty,Medical,Enrollment Status
0,132083,Male,USP Senior UG,"USP, Single (Corridor, Air-Con)","USP, Single (6 bdrm Apt, Air-Con)","USP, Single (6 bdrm Apt, Non Air-Con)",15100,Mixed Gender Floor,SINGAPORE CITIZEN,False,1510,4.0,ARTS & SOCIAL SCIENCES,NaN,CURRENT


In [4]:
class Room(object):    
    def __init__(self, floor_num, pos_id):
        self.floor_num = floor_num
        self.pos_id = pos_id
    
    def __str__(self):
        return '{}-{}'.format(self.floor_num, self.pos_id)
    
    def __eq__(self, other):
        return self.floor_num == other.floor_num and self.pos_id == other.pos_id

    def __hash__(self):
        return hash(type(self), self.floor_num, self.pos_id)
    
    __repr__ = __str__

unifiable(Room)

class Resident(object):
    def __init__(self, name, gender):
        self.name = name
        self.gender = gender
    
    def __str__(self):
        return "{} ({})".format(self.name, self.gender)
    
    __repr__ = __str__
    
unifiable(Resident)

class Configuration(object):
    def __init__(self, persons):
        self.persons = persons
        self.rooms = []

    def __str__(self):
        return "{} persons".format(len(self.persons))
    
    __repr__ = __str__
    
unifiable(Configuration)

__main__.Configuration

In [14]:
# Floors

all_floors = set(range(4, 22))
rf_floors = set([5, 9, 13, 17, 21])
freshmen_floors = set([6, 8, 11, 14, 16, 20])
male_floors = set([8, 10, 16, 19])
female_floors = set([7, 14, 18, 20])
mixed_floors = all_floors.difference(male_floors).difference(female_floors)

def floor(x):
    return membero(x, all_floors)
    
def freshmen_floors(fnum):
    return membero(x, mixed_floors)

def male_floor(x):
    return membero(x, male_floors)

def female_floor(x):
    return membero(x, female_floors)
    
def mixed_floor(fnum):
    return membero(x, mixed_floors)


# Rooms
corridor_room_pos = set(range(101, 107)).union(range(109, 121))
suite_pos = set([100, 107, 108])

def gen_rooms_from_suite(suite_num):
    return [str(suite_num) + chr(ord('A') + i) for i in range(6)]

suite_room_pos = set(itertools.chain(*map(gen_rooms_from_suite, suite_pos)))
all_room_pos = corridor_room_pos.union(suite_room_pos)

def room(r):
    floor_ok = floor(r.floor_num)
    pos_ok = membero(r.pos_id, all_room_pos)
    return conde((floor_ok, pos_ok))

# Residents

def male(res):
    return eq(res.gender, "male")

def female(res):
    return eq(res.gender, "female")

def room_match(p, r):
    gender_ok = (room(r), male(p), male_floor(r.floor_num)), (room(r), female(p), female_floor(r.floor_num))
    return conde(*gender_ok)

def configuration_ok(c):
    goals = []
    for p in c.persons:
        fn, pos = vars(2)
        r = Room(fn, pos)
        c.rooms.append(r)
        goals.append(room_match(p, r))
        
    return lall(*goals)

x = var()
y = var()
r = Room(10, y)

run(5, r, room(r))

(10-108F, 10-108E, 10-108A, 10-108D, 10-100F)